# ECS scene to GNN digestable format and scene classification example 

### Import libraries

In [ ]:
# Run the following code to install additional packages required for this example
! pip install matplotlib torch 
! pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv torch_geometric  --find-links https://data.pyg.org/whl/torch-1.12.0+cpu.html
! pip install trimesh

In [ ]:
from __future__ import annotations
import sys
sys.path.append("../../../")
import numpy as np

from Elements.pyGLV.GL.GameObject import GameObject
from Elements.pyECSS.Component import BasicTransform, RenderMesh
from Elements.pyECSS.Entity import Entity
from Elements.pyGLV.GL.Scene import Scene
from Elements.pyGLV.GL.Shader import Shader, ShaderGLDecorator
from Elements.pyGLV.GL.VertexArray import VertexArray

import numpy as np

import Elements.pyECSS.math_utilities as util
from Elements.pyECSS.Entity import Entity
from Elements.pyECSS.Component import BasicTransform, RenderMesh, Camera
from Elements.pyECSS.System import TransformSystem, CameraSystem
from Elements.pyGLV.GL.Scene import Scene
from Elements.pyGLV.GUI.Viewer import RenderGLStateSystem

from Elements.pyGLV.GL.Shader import InitGLShaderSystem, Shader, ShaderGLDecorator, RenderGLShaderSystem
from Elements.pyGLV.GL.VertexArray import VertexArray

from OpenGL.GL import GL_LINES
from CreateScenes import CreateRoomScene,CreateORScene,CreatePaperScene
import Elements.utils.Converter as Converter
import torch


### Function that creates a default living room scene.

In [ ]:
Scene.reset_instance()
scene = CreateRoomScene(visualize=True)

### Function that creates a default operating room (OR) scene.

In [ ]:
Scene.reset_instance()
scene = CreateORScene(visualize=True)

### Function that creates the OR scene as seen in the SIGGRAPH poster

In [ ]:
Scene.reset_instance()
scene = CreatePaperScene(visualize=True)

### Create the instances of the two default scenes, and convert them into a pytorch geometric graph format and also add the corresponding label to it

In [ ]:
Scene.reset_instance()
defaultRoomScene = CreateRoomScene()
defaultRoomSceneGNN = Converter.ECStoGNN(defaultRoomScene)
defaultRoomSceneGNN.y = 1
Scene.reset_instance()
defaultORScene = CreateORScene()
defaultORSceneGNN = Converter.ECStoGNN(defaultORScene)
defaultORSceneGNN.y = 0
Scene.reset_instance()
mydata = []
y = []

### Create 10 scenes, 5 of each room. The function ECStoGNN adds noise to each scene's component data so we get unique scenes each time. Finally we save them to a list

In [ ]:
numscenes = 10
for i in range(numscenes):
    print("I:", i)
    Scene.reset_instance()
    if i > numscenes / 2:
        y.append(0)
        scene = CreateORScene()
    else:
        y.append(1)
        scene = CreateRoomScene()

    data = Converter.ECStoGNN(scene)
    data.y = y[len(y) - 1]
    mydata.append(data)

mydata.append(defaultRoomSceneGNN)
mydata.append(defaultORSceneGNN)

### Create the GNN classifier's architecture. It consists of hetero convolutional layers that apply the SAGEConv operator on each type of edge. More information about the SAGEConv operator are here: 
https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.SAGEConv.html

In [ ]:
from torch_geometric.nn import SAGEConv, HeteroConv
from torch_geometric.nn import global_mean_pool
from torch.nn import Linear
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## GNN Classifier architecture
class HeteroGNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_layers):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                ('entity', 'entparent', 'entity'): SAGEConv((-1, -1), hidden_channels, normalize=True),
                ('entity', 'trsparent', 'trs'): SAGEConv((-1, -1), hidden_channels, normalize=True),
                ('entity', 'meshparent', 'mesh'): SAGEConv((-1, -1), hidden_channels, normalize=True),
            }, aggr='mean')
            self.convs.append(conv)

        self.lin1 = Linear(hidden_channels, out_channels)
        self.lin2 = Linear(hidden_channels, out_channels)
        self.lin3 = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict, batch1, batch2, batch3):
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: x.relu() for key, x in x_dict.items()}
        x1 = global_mean_pool(x_dict['entity'], batch1)  # [batch_size, hidden_channels]
        x2 = global_mean_pool(x_dict['trs'], batch2)  # [batch_size, hidden_channels]
        x3 = global_mean_pool(x_dict['mesh'], batch3)  # [batch_size, hidden_channels]
        x1 = self.lin1(x1)
        x2 = self.lin2(x2)
        x3 = self.lin3(x3)
        final = x1 + x2 + x3
        final = torch.sigmoid(final)
        return final


model = HeteroGNN(hidden_channels=128, out_channels=2,
                  num_layers=5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()
# random.shuffle(mydata)

### Split the data into training and test set and train for 50 epochs. Calculate the loss and accuracy for each epoch on the test and training set. Finally, save the model

In [ ]:
from torch_geometric.loader import DataLoader
train_data = mydata[:int(numscenes * 0.7)]
test_data = mydata[int(numscenes * 0.7):]
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1, shuffle=True)

# Train for 50 epochs and print losses and accuracy
for i in range(50):
    correct = 0
    totalloss = 0
    model.train()
    for l in train_loader:
        a = model(l.x_dict, l.edge_index_dict, l['entity'].batch, l['trs'].batch, l['mesh'].batch)
        pred = a.argmax(dim=1).to(device)
       
        correct += int((pred == l.y.to(device)).sum())
        loss = criterion(a, l.y.to(device))  # Compute the loss.
        totalloss += loss.item()
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    model.eval()
    testcorrect = 0
    for l in test_loader:
        a = model(l.x_dict, l.edge_index_dict, l['entity'].batch, l['trs'].batch, l['mesh'].batch)
        pred = a.argmax(dim=1).to(device)
        # print(l.y.shape)
        testcorrect += int((pred == l.y.to(device)).sum())
    print(
        f'Epoch: {i:03d}, Train Acc: {correct / len(train_loader.dataset) :.4f}, Test Acc: {testcorrect / len(test_loader.dataset)} Loss: {totalloss / 8:.4f}')

torch.save(model.state_dict(), "scenemodel.pth")